In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

import os
import numpy as np
from pandas import DataFrame
import pandas as pd
import re
from notebook_utils import NotebookUtilities
from datetime import timedelta
import matplotlib.pyplot as plt
import re

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))


# OSU dataset of Simulation Sessions, deidentified

Doug screened all of the **VOICE_COMMAND** and **VOICE_CAPTURE** lines and replaced any names with either Max or Jane, regardless of whether the name was that of the trainee.

In [3]:

# Group the event times by consecutive minutes
def get_datetime_indexes(df):
    by_minute = timedelta(minutes=1)
    
    # Create a list of time stamps
    time_stamps = sorted(df.event_time.unique())
    
    # Calculate the time difference between consecutive time stamps
    time_diffs = [b - a for a, b in zip(time_stamps[:-1], time_stamps[1:])]
    
    # Check if any of the time differences are larger than a minute
    if any(time_diff > by_minute for time_diff in time_diffs):
        
        # Create a list of groups
        groups = []
        current_group = []
        for time_stamp, time_diff in zip(time_stamps, time_diffs):
            current_group.append(time_stamp)
            if time_diff > by_minute:
                groups.append(current_group)
                current_group = []
        
        # Add the last group to the list of groups
        current_group.append(time_stamps[-1])
        groups.append(current_group)
            
    else:
        groups = [time_stamps]
    
    # Create a list of DatetimeIndexes
    datetime_indexes_list = []
    for stamps_list in groups:
        start_date = min(stamps_list)
        end_date = max(stamps_list)
        datetime_index = pd.date_range(start=start_date, end=end_date, freq=by_minute)
        datetime_indexes_list.append(datetime_index)

    return datetime_indexes_list

In [68]:

def set_mcivr_metrics_types(action_type, frvrs_logs_df, row_index, row_series):
    if (action_type == 'BAG_ACCESS'): # BagAccess
        frvrs_logs_df.loc[row_index, 'bag_access_location'] = row_series[4] # Location
    elif (action_type == 'BAG_CLOSED'): # BagClosed
        frvrs_logs_df.loc[row_index, 'bag_closed_location'] = row_series[4] # Location
    elif (action_type == 'INJURY_RECORD'): # InjuryRecord
        frvrs_logs_df.loc[row_index, 'injury_record_id'] = row_series[4] # Id
        frvrs_logs_df.loc[row_index, 'injury_record_patient_id'] = row_series[5] # patientId
        frvrs_logs_df.loc[row_index, 'injury_record_required_procedure'] = row_series[6] # requiredProcedure
        frvrs_logs_df.loc[row_index, 'injury_record_severity'] = row_series[7] # severity
        frvrs_logs_df.loc[row_index, 'injury_record_body_region'] = row_series[8] # bodyRegion
        frvrs_logs_df.loc[row_index, 'injury_record_injury_treated'] = row_series[9] # injuryTreated
        frvrs_logs_df.loc[row_index, 'injury_record_injury_treated_with_wrong_treatment'] = row_series[10] # injuryTreatedWithWrongTreatment
        frvrs_logs_df.loc[row_index, 'injury_record_injury_injury_locator'] = row_series[11] # injuryLocator
    elif (action_type == 'INJURY_TREATED'): # InjuryTreated
        frvrs_logs_df.loc[row_index, 'injury_treated_id'] = row_series[4] # Id
        frvrs_logs_df.loc[row_index, 'injury_treated_patient_id'] = row_series[5] # patientId
        frvrs_logs_df.loc[row_index, 'injury_treated_required_procedure'] = row_series[6] # requiredProcedure
        frvrs_logs_df.loc[row_index, 'injury_treated_severity'] = row_series[7] # severity
        frvrs_logs_df.loc[row_index, 'injury_treated_body_region'] = row_series[8] # bodyRegion
        frvrs_logs_df.loc[row_index, 'injury_treated_injury_treated'] = row_series[9] # injuryTreated
        frvrs_logs_df.loc[row_index, 'injury_treated_injury_treated_with_wrong_treatment'] = row_series[10] # injuryTreatedWithWrongTreatment
        frvrs_logs_df.loc[row_index, 'injury_treated_injury_injury_locator'] = row_series[11] # injuryLocator
    elif (action_type == 'PATIENT_DEMOTED'): # PatientDemoted
        frvrs_logs_df.loc[row_index, 'patient_demoted_health_level'] = row_series[4] # healthLevel
        frvrs_logs_df.loc[row_index, 'patient_demoted_health_time_remaining'] = row_series[5] # healthTimeRemaining
        frvrs_logs_df.loc[row_index, 'patient_demoted_id'] = row_series[6] # id
        frvrs_logs_df.loc[row_index, 'patient_demoted_position'] = row_series[7] # position
        frvrs_logs_df.loc[row_index, 'patient_demoted_rotation'] = row_series[8] # rotation
        frvrs_logs_df.loc[row_index, 'patient_demoted_salt'] = row_series[9] # salt
        frvrs_logs_df.loc[row_index, 'patient_demoted_sort'] = row_series[10] # sort
        frvrs_logs_df.loc[row_index, 'patient_demoted_pulse'] = row_series[11] # pulse
        frvrs_logs_df.loc[row_index, 'patient_demoted_breath'] = row_series[12] # breath
        frvrs_logs_df.loc[row_index, 'patient_demoted_hearing'] = row_series[13] # hearing
        frvrs_logs_df.loc[row_index, 'patient_demoted_mood'] = row_series[14] # mood
        frvrs_logs_df.loc[row_index, 'patient_demoted_pose'] = row_series[15] # pose
    elif (action_type == 'PATIENT_ENGAGED'): # PatientEngaged
        frvrs_logs_df.loc[row_index, 'patient_engaged_health_level'] = row_series[4] # healthLevel
        frvrs_logs_df.loc[row_index, 'patient_engaged_health_time_remaining'] = row_series[5] # healthTimeRemaining
        frvrs_logs_df.loc[row_index, 'patient_engaged_id'] = row_series[6] # id
        frvrs_logs_df.loc[row_index, 'patient_engaged_position'] = row_series[7] # position
        frvrs_logs_df.loc[row_index, 'patient_engaged_rotation'] = row_series[8] # rotation
        frvrs_logs_df.loc[row_index, 'patient_engaged_salt'] = row_series[9] # salt
        frvrs_logs_df.loc[row_index, 'patient_engaged_sort'] = row_series[10] # sort
        frvrs_logs_df.loc[row_index, 'patient_engaged_pulse'] = row_series[11] # pulse
        frvrs_logs_df.loc[row_index, 'patient_engaged_breath'] = row_series[12] # breath
        frvrs_logs_df.loc[row_index, 'patient_engaged_hearing'] = row_series[13] # hearing
        frvrs_logs_df.loc[row_index, 'patient_engaged_mood'] = row_series[14] # mood
        frvrs_logs_df.loc[row_index, 'patient_engaged_pose'] = row_series[15] # pose
    elif (action_type == 'PATIENT_RECORD'): # PatientRecord
        frvrs_logs_df.loc[row_index, 'patient_record_health_level'] = row_series[4] # healthLevel
        frvrs_logs_df.loc[row_index, 'patient_record_health_time_remaining'] = row_series[5] # healthTimeRemaining
        frvrs_logs_df.loc[row_index, 'patient_record_id'] = row_series[6] # id
        frvrs_logs_df.loc[row_index, 'patient_record_position'] = row_series[7] # position
        frvrs_logs_df.loc[row_index, 'patient_record_rotation'] = row_series[8] # rotation
        frvrs_logs_df.loc[row_index, 'patient_record_salt'] = row_series[9] # salt
        frvrs_logs_df.loc[row_index, 'patient_record_sort'] = row_series[10] # sort
        frvrs_logs_df.loc[row_index, 'patient_record_pulse'] = row_series[11] # pulse
        frvrs_logs_df.loc[row_index, 'patient_record_breath'] = row_series[12] # breath
        frvrs_logs_df.loc[row_index, 'patient_record_hearing'] = row_series[13] # hearing
        frvrs_logs_df.loc[row_index, 'patient_record_mood'] = row_series[14] # mood
        frvrs_logs_df.loc[row_index, 'patient_record_pose'] = row_series[15] # pose
    elif (action_type == 'PULSE_TAKEN'): # PulseTaken
        frvrs_logs_df.loc[row_index, 'pulse_taken_pulse_name'] = row_series[4] # pulseName
        frvrs_logs_df.loc[row_index, 'pulse_taken_patient_id'] = row_series[5] # patientId
    elif (action_type == 'S_A_L_T_WALKED'): # SALTWalked
        frvrs_logs_df.loc[row_index, 's_a_l_t_walked_sort_location'] = row_series[4] # sortLocation
        frvrs_logs_df.loc[row_index, 's_a_l_t_walked_sort_command_text'] = row_series[5] # sortCommandText
        frvrs_logs_df.loc[row_index, 's_a_l_t_walked_patient_id'] = row_series[6] # patientId
    elif (action_type == 'S_A_L_T_WALK_IF_CAN'): # SALTWalkIfCan
        frvrs_logs_df.loc[row_index, 's_a_l_t_walk_if_can_sort_location'] = row_series[4] # sortLocation
        frvrs_logs_df.loc[row_index, 's_a_l_t_walk_if_can_sort_command_text'] = row_series[5] # sortCommandText
        frvrs_logs_df.loc[row_index, 's_a_l_t_walk_if_can_patient_id'] = row_series[6] # patientId
    elif (action_type == 'S_A_L_T_WAVED'): # SALTWave
        frvrs_logs_df.loc[row_index, 's_a_l_t_waved_sort_location'] = row_series[4] # sortLocation
        frvrs_logs_df.loc[row_index, 's_a_l_t_waved_sort_command_text'] = row_series[5] # sortCommandText
        frvrs_logs_df.loc[row_index, 's_a_l_t_waved_patient_id'] = row_series[6] # patientId
    elif (action_type == 'S_A_L_T_WAVE_IF_CAN'): # SALTWaveIfCan
        frvrs_logs_df.loc[row_index, 's_a_l_t_wave_if_can_sort_location'] = row_series[4] # sortLocation
        frvrs_logs_df.loc[row_index, 's_a_l_t_wave_if_can_sort_command_text'] = row_series[5] # sortCommandText
        frvrs_logs_df.loc[row_index, 's_a_l_t_wave_if_can_patient_id'] = row_series[6] # patientId
    elif (action_type == 'TAG_APPLIED'): # TagApplied
        frvrs_logs_df.loc[row_index, 'tag_applied_patient_id'] = row_series[4] # patientId
        frvrs_logs_df.loc[row_index, 'tag_applied_type'] = row_series[5] # type
    elif (action_type == 'TAG_DISCARDED'): # TagDiscarded
        frvrs_logs_df.loc[row_index, 'tag_discarded_type'] = row_series[4] # Type
        frvrs_logs_df.loc[row_index, 'tag_discarded_location'] = row_series[5] # Location
    elif (action_type == 'TAG_SELECTED'): # TagSelected
        frvrs_logs_df.loc[row_index, 'tag_selected_type'] = row_series[4] # Type
    elif (action_type == 'TELEPORT'): # Teleport
        frvrs_logs_df.loc[row_index, 'teleport_location'] = row_series[4] # Location
    elif (action_type == 'TOOL_APPLIED'): # ToolApplied
        tool_applied_patient_id = row_series[4]
        if ' Root' in tool_applied_patient_id:
            frvrs_logs_df.loc[row_index, 'tool_applied_patient_id'] = tool_applied_patient_id # patientId
        frvrs_logs_df.loc[row_index, 'tool_applied_type'] = row_series[5] # type
        frvrs_logs_df.loc[row_index, 'tool_applied_attachment_point'] = row_series[6] # attachmentPoint
        frvrs_logs_df.loc[row_index, 'tool_applied_tool_location'] = row_series[7] # toolLocation
        frvrs_logs_df.loc[row_index, 'tool_applied_data'] = row_series[8] # data
        frvrs_logs_df.loc[row_index, 'tool_applied_sender'] = row_series[9] # sender
        frvrs_logs_df.loc[row_index, 'tool_applied_attach_message'] = row_series[10] # attachMessage
    elif (action_type == 'TOOL_DISCARDED'): # ToolDiscarded
        frvrs_logs_df.loc[row_index, 'tool_discarded_type'] = row_series[4] # Type
        frvrs_logs_df.loc[row_index, 'tool_discarded_count'] = row_series[5] # Count
        frvrs_logs_df.loc[row_index, 'tool_discarded_location'] = row_series[6] # Location
    elif (action_type == 'TOOL_HOVER'): # ToolHover
        frvrs_logs_df.loc[row_index, 'tool_hover_type'] = row_series[4] # Type
        frvrs_logs_df.loc[row_index, 'tool_hover_count'] = row_series[5] # Count
    elif (action_type == 'TOOL_SELECTED'): # ToolSelected
        frvrs_logs_df.loc[row_index, 'tool_selected_type'] = row_series[4] # Type
        frvrs_logs_df.loc[row_index, 'tool_selected_count'] = row_series[5] # Count
    elif (action_type == 'VOICE_CAPTURE'): # VoiceCapture
        frvrs_logs_df.loc[row_index, 'voice_capture_message'] = row_series[4] # Message
        frvrs_logs_df.loc[row_index, 'voice_capture_command_description'] = row_series[5] # commandDescription
    elif (action_type == 'VOICE_COMMAND'): # VoiceCommand
        frvrs_logs_df.loc[row_index, 'voice_command_message'] = row_series[4] # Message
        frvrs_logs_df.loc[row_index, 'voice_command_command_description'] = row_series[5] # commandDescription
    elif (action_type == 'PLAYER_LOCATION'): # PlayerLocation
        frvrs_logs_df.loc[row_index, 'player_location_location'] = row_series[4] # Location
    elif (action_type == 'PLAYER_GAZE'): # PlayerGaze
        frvrs_logs_df.loc[row_index, 'player_gaze_location'] = row_series[4] # Location
        frvrs_logs_df.loc[row_index, 'player_gaze_patient_id'] = row_series[5] # patientId
        frvrs_logs_df.loc[row_index, 'player_gaze_distance'] = row_series[6] # distance
        frvrs_logs_df.loc[row_index, 'player_gaze_rotation'] = row_series[7] # rotation

    return frvrs_logs_df

In [73]:

def concatonate_logs(logs_folder='../data/logs'):
    frvrs_logs_df = DataFrame([])
    for sub_directory, directories_list, files_list in os.walk(logs_folder):
        for file_name in files_list:
            if sub_directory.endswith('v.1.0'):
                file_filter = file_name.endswith('.csv') and file_name.lower().startswith('clean')
            elif sub_directory.endswith('v.1.3'):
                file_filter = file_name.endswith('.csv')
            if file_filter:
                file_path = os.path.join(sub_directory, file_name)
                if sub_directory.endswith('v.1.0'):
                    df = pd.read_csv(file_path, header=None, index_col=False)
                else:
                    version_number = '1.3'
                    rows_list = []
                    with open(file_path, 'r') as f:
                        reader = csv.reader(f, delimiter=',', quotechar='"')
                        for values_list in reader:
                            version_number = values_list.pop(4) # Remove version number column
                            if (values_list[-1] == ''): values_list.pop(-1)
                            rows_list.append({i: v for i, v in enumerate(values_list)})
                    df = pd.DataFrame(rows_list)
                if sub_directory.endswith('v.1.0'):
                    df['file_name'] = file_name.replace('.csv', '').replace('clean-', '').replace('Clean ', '')
                    df['logger_version'] = 1.0
                else:
                    df['file_name'] = file_name.replace('.csv', '')
                    df['logger_version'] = float(version_number)
                frvrs_logs_df = pd.concat([frvrs_logs_df, df], axis='index')
    frvrs_logs_df = frvrs_logs_df.reset_index(drop=True)

    return frvrs_logs_df

In [69]:

def set_time_groups(frvrs_logs_df):

    # Section off player actions by session start and end
    old_session_uuid = ''
    gaze_rows_list = []
    for row_index, row_series in frvrs_logs_df.iterrows():
        action_type = row_series.action_type
        # elapsed_time = row_series.elapsed_time
        # event_time = row_series.event_time
        new_session_uuid = row_series.session_uuid
        if new_session_uuid != old_session_uuid:
            time_group = 0
            old_session_uuid = new_session_uuid
        if (action_type == 'SESSION_END'): # SessionEnd
            frvrs_logs_df.loc[row_index, 'time_group'] = time_group
            continue
        elif (action_type == 'SESSION_START'): # SessionStart
            time_group += 1
            frvrs_logs_df.loc[row_index, 'time_group'] = time_group
            continue
        frvrs_logs_df.loc[row_index, 'time_group'] = time_group
        frvrs_logs_df = set_mcivr_metrics_types(action_type, frvrs_logs_df, row_index, row_series)
        if (action_type in ['PLAYER_GAZE', 'PLAYER_LOCATION']):
            gaze_rows_list.append(row_series)
    
    # Add time groups
    mask_series = frvrs_logs_df.time_group.isnull()
    if frvrs_logs_df[mask_series].shape[0]:
        columns_list = ['session_uuid', 'elapsed_time', 'event_time']
        srs = frvrs_logs_df[columns_list].groupby(['session_uuid']).apply(get_datetime_indexes)
        for session_uuid in srs.index:
            datetime_indexes_list = srs[session_uuid]
            for i, datetime_index in enumerate(datetime_indexes_list):
                mask_series = (frvrs_logs_df.session_uuid == session_uuid) & frvrs_logs_df.event_time.isin(datetime_index)
                frvrs_logs_df.loc[mask_series, 'time_group'] = frvrs_logs_df[mask_series].time_group.mode()
                print(frvrs_logs_df[mask_series].time_group.dropna())
    frvrs_logs_df.time_group = frvrs_logs_df.time_group.astype('int64')

    return gaze_rows_list, frvrs_logs_df

In [74]:

def modalize_patient_id(frvrs_logs_df):
    columns_list= [
        'patient_demoted_id', 'patient_record_id', 'injury_record_patient_id', 's_a_l_t_walk_if_can_patient_id',
        's_a_l_t_walked_patient_id', 's_a_l_t_wave_if_can_patient_id', 's_a_l_t_waved_patient_id', 'patient_engaged_id',
        'pulse_taken_patient_id', 'injury_treated_patient_id', 'tool_applied_patient_id', 'tag_applied_patient_id',
        'player_gaze_patient_id'
    ]
    mask_series = (frvrs_logs_df[columns_list].apply(pd.Series.nunique, axis='columns') == 1)
    frvrs_logs_df.loc[~mask_series, 'patient_id'] = np.nan
    def f(srs):
        cn = srs.first_valid_index()
        
        return srs[cn]
    frvrs_logs_df.loc[mask_series, 'patient_id'] = frvrs_logs_df[mask_series][columns_list].apply(f, axis='columns')

    return frvrs_logs_df

In [75]:

def modalize_location_id(frvrs_logs_df):
    
    # All locations have zero in the y dimension and positive floats in the x and z dimensions
    # location_regex = re.compile(r'\((-?\d\.\d, ){2}-?\d\.\d\)')
    # srs = frvrs_logs_df.applymap(lambda x: bool(location_regex.fullmatch(str(x))), na_action='ignore').sum()
    # mask_series = (srs != 0)
    # columns_list = srs[mask_series].index.tolist()
    
    # Modalize into one location ID column if possible
    columns_list= [
        'teleport_location', 'patient_demoted_position', 'patient_record_position', 'injury_record_injury_injury_locator',
        's_a_l_t_walk_if_can_sort_location', 's_a_l_t_walked_sort_location', 's_a_l_t_wave_if_can_sort_location',
        's_a_l_t_waved_sort_location', 'patient_engaged_position', 'bag_access_location', 'injury_treated_injury_injury_locator',
        'bag_closed_location', 'tag_discarded_location', 'tool_discarded_location', 'player_location_location',
        'player_gaze_location'
    ]
    mask_series = (frvrs_logs_df[columns_list].apply(pd.Series.nunique, axis='columns') == 1)
    frvrs_logs_df.loc[~mask_series, 'location_id'] = np.nan
    def f(srs):
        cn = srs.first_valid_index()
        
        return srs[cn]
    frvrs_logs_df.loc[mask_series, 'location_id'] = frvrs_logs_df[mask_series][columns_list].apply(f, axis='columns')

    return frvrs_logs_df 

In [76]:

import csv

# Get all logs into one DataFrame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
else:

    # Add the CSVs to the data frame
    frvrs_logs_df = concatonate_logs()
        
    # Name the global columns
    columns_list = ['action_type', 'elapsed_time', 'event_time', 'session_uuid']
    frvrs_logs_df.columns = columns_list + frvrs_logs_df.columns.tolist()[len(columns_list):]
        
    # Parse the date column
    if sub_directory.endswith('v.1.0'): frvrs_logs_df.event_time = pd.to_datetime(frvrs_logs_df.event_time, format='%m/%d/%Y %H:%M')
    # elif sub_directory.endswith('v.1.3'): frvrs_logs_df.event_time = pd.to_datetime(frvrs_logs_df.event_time, format='%m/%d/%Y %I:%M:%S %p')
    else: frvrs_logs_df.event_time = pd.to_datetime(frvrs_logs_df.event_time, format='mixed')

    # Section off player actions by session start and end
    gaze_rows_list, frvrs_logs_df = set_time_groups(frvrs_logs_df)

    # Remove numeric columns
    columns_list = [x for x in frvrs_logs_df.columns if not re.search(r'\d+', str(x))]
    frvrs_logs_df = frvrs_logs_df[columns_list]
    
    # Convert 'TRUE' and 'FALSE' to boolean values
    for cn in [
        'injury_record_injury_treated_with_wrong_treatment', 'injury_record_injury_treated',
        'injury_treated_injury_treated_with_wrong_treatment', 'injury_treated_injury_treated'
    ]:
        frvrs_logs_df[cn] = frvrs_logs_df[cn].map({'TRUE': True, 'FALSE': False})
    
    # Modalize into one patient ID column if possible
    frvrs_logs_df = modalize_patient_id(frvrs_logs_df)
    
    # Modalize into one location ID column if possible
    frvrs_logs_df = modalize_location_id(frvrs_logs_df)
    
    # nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    # nu.save_dataframes(frvrs_logs_df=frvrs_logs_df)
print(frvrs_logs_df.shape) # (168528, 106)
df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
df.T

(168528, 106)


,10299,81352,137788,109017
action_type,VOICE_CAPTURE,S_A_L_T_WALK_IF_CAN,TOOL_HOVER,TOOL_HOVER
elapsed_time,142089,176922,394776,521690
event_time,2023-03-08 09:55:00,2023-05-09 11:00:00,2023-04-21 12:32:00,2023-06-07 09:31:00
session_uuid,5254234d-5334-44d0-a97b-d0e098c129a9,868cbf3b-f04a-4e1a-a207-95881d529378,3bac8365-232a-4be0-a551-c5214b9f0706,b440a9ce-e511-43e3-bc20-c9b2bed11005
file_name,5254234d-5334-44d0-a97b-d0e098c129a9,868cbf3b-f04a-4e1a-a207-95881d529378,max15,b440a9ce-e511-43e3-bc20-c9b2bed11005
logger_version,1.0,1.0,1.0,1.0
time_group,1,1,1,2
voice_capture_message,i don't know this wait does it turn yellow the...,NaN,NaN,NaN
s_a_l_t_walk_if_can_sort_location,NaN,"(4.5, 0.0, -1.8)",NaN,NaN
s_a_l_t_walk_if_can_sort_command_text,NaN,walkIfCan,NaN,NaN


In [78]:

# Find location columns that haven't been collapsed into location ID
location_regex = re.compile(r'\((-?\d\.\d, ){2}-?\d\.\d\)')
srs = frvrs_logs_df.applymap(lambda x: bool(location_regex.fullmatch(str(x))), na_action='ignore').sum()
mask_series = (srs != 0)
location_columns_list = srs[mask_series].index.tolist()
mask_series = False
for cn in location_columns_list:
    mask_series |= ~frvrs_logs_df[cn].isnull()
mask_series &= frvrs_logs_df.location_id.isnull()
df = frvrs_logs_df[mask_series].sample(min(4, frvrs_logs_df[mask_series].shape[0])).dropna(axis='columns', how='all')
df.T

""


In [84]:

# Prove that all player locations have zero in the y dimension
def f(x):
    filter = False
    if location_regex.fullmatch(str(x)):
        filter = (eval(x)[1] != 0.0)

    return filter
nonzero_list = []
for cn in location_columns_list:
    if any(frvrs_logs_df[cn].map(f)):
        nonzero_list.append(cn)
print(nonzero_list)
columns_list= [
    'teleport_location', 'patient_demoted_position', 'patient_record_position', 'injury_record_injury_injury_locator',
    's_a_l_t_walk_if_can_sort_location', 's_a_l_t_walked_sort_location', 's_a_l_t_wave_if_can_sort_location',
    's_a_l_t_waved_sort_location', 'patient_engaged_position', 'bag_access_location', 'injury_treated_injury_injury_locator',
    'bag_closed_location', 'tag_discarded_location', 'tool_discarded_location', 'player_location_location',
    'player_gaze_location'
]
set(nonzero_list) - set(columns_list)

['teleport_location', 'patient_demoted_position', 'patient_record_position', 'injury_record_injury_injury_locator', 's_a_l_t_walk_if_can_sort_location', 's_a_l_t_walked_sort_location', 's_a_l_t_wave_if_can_sort_location', 's_a_l_t_waved_sort_location', 'patient_engaged_position', 'bag_access_location', 'injury_treated_injury_injury_locator', 'bag_closed_location', 'tag_discarded_location', 'tool_discarded_location', 'player_location_location', 'player_gaze_rotation', 'location_id']


{'player_gaze_rotation', 'location_id'}

In [90]:

def f(x):
    filter = False
    if location_regex.fullmatch(str(x)):
        filter = (eval(x)[1] != 0.0)

    return filter
for cn in nonzero_list:
    mask_series = frvrs_logs_df[cn].map(f)
    df = frvrs_logs_df[mask_series].sample(min(4, frvrs_logs_df[mask_series].shape[0])).dropna(axis='columns', how='all')
    print(cn)
    display(df.T)

teleport_location


,168505,168504,167041,167259
action_type,TELEPORT,TELEPORT,TELEPORT,TELEPORT
elapsed_time,240135,208133,259082,407325
event_time,2023-09-07 08:57:59,2023-09-07 08:57:27,2023-09-07 15:37:31,2023-09-07 15:39:59
session_uuid,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0,f8b5cbd1-0abe-483b-8b16-956044dc0119,f8b5cbd1-0abe-483b-8b16-956044dc0119
file_name,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0,f8b5cbd1-0abe-483b-8b16-956044dc0119,f8b5cbd1-0abe-483b-8b16-956044dc0119
logger_version,1.3,1.3,1.3,1.3
time_group,2,2,1,1
teleport_location,"(0.9, 0.1, -7.6)","(0.9, 0.1, -7.6)","(3.6, 0.5, -9.2)","(3.6, 0.5, -9.2)"
location_id,"(0.9, 0.1, -7.6)","(0.9, 0.1, -7.6)","(3.6, 0.5, -9.2)","(3.6, 0.5, -9.2)"


patient_demoted_position


,166266,166686,154891,167401
action_type,PATIENT_DEMOTED,PATIENT_DEMOTED,PATIENT_DEMOTED,PATIENT_DEMOTED
elapsed_time,7512,11861,18360,8960
event_time,2023-09-07 13:55:31,2023-09-07 15:33:23,2023-09-07 12:52:25,2023-09-07 14:49:02
session_uuid,ebc4fb38-01c3-4296-b22d-34adf70e0d43,f8b5cbd1-0abe-483b-8b16-956044dc0119,8b979d02-dbea-4d22-864b-7031425815cb,fde2c201-36fc-4123-bf42-f1bff1b24969
file_name,ebc4fb38-01c3-4296-b22d-34adf70e0d43,f8b5cbd1-0abe-483b-8b16-956044dc0119,8b979d02-dbea-4d22-864b-7031425815cb,fde2c201-36fc-4123-bf42-f1bff1b24969
logger_version,1.3,1.3,1.3,1.3
time_group,0,0,0,0
patient_demoted_health_level,100,100,100,100
patient_demoted_health_time_remaining,128,128,128,128
patient_demoted_id,Military Mike Jungle Scout_1_3 Root,Military Mike Jungle Scout_1_3 Root,Military Mike Jungle Scout_1_3 Root,Military Mike Jungle Scout_1_3 Root


patient_record_position


...

,151771,149689,153772,162878
action_type,PLAYER_LOCATION,PLAYER_LOCATION,PLAYER_LOCATION,PLAYER_LOCATION
elapsed_time,167718,217890,41025,199280
event_time,2023-09-11 12:48:32,2023-09-07 14:45:13,2023-09-07 15:28:10,2023-09-11 13:06:07
session_uuid,2a5106e4-984c-42a7-9edd-2fb3e6325d10,0b630a14-5acd-4e11-99df-8f59d804f807,666ce61c-2ebc-40ee-902e-6f6aa42801ad,d0f826d3-a6db-4a6a-9ceb-5e17e3ff3d17
file_name,2a5106e4-984c-42a7-9edd-2fb3e6325d10,0b630a14-5acd-4e11-99df-8f59d804f807,666ce61c-2ebc-40ee-902e-6f6aa42801ad,d0f826d3-a6db-4a6a-9ceb-5e17e3ff3d17
logger_version,1.3,1.3,1.3,1.3
time_group,1,1,1,1
player_location_location,"(2.2, 1.6, 2.1)","(2.5, 1.6, 1.9)","(0.5, 1.9, 2.7)","(2.2, 1.5, 2.2)"
location_id,"(2.2, 1.6, 2.1)","(2.5, 1.6, 1.9)","(0.5, 1.9, 2.7)","(2.2, 1.5, 2.2)"


player_gaze_rotation


,167757,165124,164930,165176
action_type,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE
elapsed_time,89045,240707,199056,248874
event_time,2023-09-07 14:50:22,2023-09-11 13:49:06,2023-09-11 13:48:24,2023-09-11 13:49:14
session_uuid,fde2c201-36fc-4123-bf42-f1bff1b24969,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,d9d58da9-9bdc-41ea-90fe-4c84db4635d9
file_name,fde2c201-36fc-4123-bf42-f1bff1b24969,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,d9d58da9-9bdc-41ea-90fe-4c84db4635d9
logger_version,1.3,1.3,1.3,1.3
time_group,1,1,1,1
player_gaze_location,"(-4.2, 0.2, -21.5)","(2.0, 0.0, 7.0)","(0.0, 0.0, 30.0)","(2.0, 0.0, 7.0)"
player_gaze_patient_id,Military Mike Jungle Combat_3_2 Root,Bob_1 Root,Lily_15 Root,Bob_1 Root
player_gaze_distance,1.522277,1.371734,1.014763,1.767851


location_id


,1702,19147,1960,168348
action_type,INJURY_RECORD,BAG_CLOSED,BAG_ACCESS,PLAYER_LOCATION
elapsed_time,1273710,4294123,1397804,79115
event_time,2023-05-16 15:20:00,2023-05-16 17:26:00,2023-05-16 15:22:00,2023-09-07 08:55:18
session_uuid,06574b6f-ab02-432c-9a65-7b031218a270,724fdf45-3165-43b8-b9ca-dc07102d2886,06574b6f-ab02-432c-9a65-7b031218a270,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0
file_name,06574b6f-ab02-432c-9a65-7b031218a270,724fdf45-3165-43b8-b9ca-dc07102d2886,06574b6f-ab02-432c-9a65-7b031218a270,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0
logger_version,1.0,1.0,1.0,1.3
time_group,5,11,5,1
injury_record_id,Forehead Scrape,NaN,NaN,NaN
injury_record_patient_id,Helga_10 Root,NaN,NaN,NaN
injury_record_required_procedure,none,NaN,NaN,NaN


In [85]:

df = DataFrame(gaze_rows_list)
df.sample(4).dropna(axis='columns', how='all').T

,160022,159723,156247,162881
action_type,PLAYER_LOCATION,PLAYER_LOCATION,PLAYER_LOCATION,PLAYER_GAZE
elapsed_time,109624,50071,204380,199797
event_time,2023-09-11 10:12:12,2023-09-11 10:11:13,2023-09-11 14:20:36,2023-09-11 13:06:07
session_uuid,b11f14c2-8de9-4247-9753-1434cb392804,b11f14c2-8de9-4247-9753-1434cb392804,a3d6d913-7755-4e8d-a174-d5e491c4eac7,d0f826d3-a6db-4a6a-9ceb-5e17e3ff3d17
4,"(1.9, 1.6, 1.5)","(2.0, 1.7, 1.5)","(2.4, 1.5, 2.0)","(2.9, 0.0, 2.5)"
5,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)",Tutorial Military Marine Root
6,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)",1.545736
7,NaN,NaN,NaN,"(-0.6, 1.4, -0.1)"
file_name,b11f14c2-8de9-4247-9753-1434cb392804,b11f14c2-8de9-4247-9753-1434cb392804,a3d6d913-7755-4e8d-a174-d5e491c4eac7,d0f826d3-a6db-4a6a-9ceb-5e17e3ff3d17
logger_version,1.3,1.3,1.3,1.3


In [86]:

mask_series = (df.action_type == 'PLAYER_LOCATION')
columns_list = [5, 6]
df[mask_series][columns_list].drop_duplicates()

,5,6
149166,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"


In [89]:

mask_series = (frvrs_logs_df.logger_version == 1.3)
df = frvrs_logs_df[mask_series]
df.sample(min(4, frvrs_logs_df[mask_series].shape[0])).dropna(axis='columns', how='all').T

,152832,165428,151369,165188
action_type,PLAYER_LOCATION,TOOL_HOVER,TOOL_HOVER,PLAYER_GAZE
elapsed_time,20422,300282,111123,251908
event_time,2023-09-07 12:43:31,2023-09-11 13:50:05,2023-09-11 12:47:35,2023-09-11 13:49:17
session_uuid,384462fc-969c-42cd-944f-726634faba4f,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,2a5106e4-984c-42a7-9edd-2fb3e6325d10,d9d58da9-9bdc-41ea-90fe-4c84db4635d9
file_name,384462fc-969c-42cd-944f-726634faba4f,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,2a5106e4-984c-42a7-9edd-2fb3e6325d10,d9d58da9-9bdc-41ea-90fe-4c84db4635d9
logger_version,1.3,1.3,1.3,1.3
time_group,0,1,1,1
tool_hover_type,NaN,Needle,Gauze,NaN
tool_hover_count,NaN,997,999,NaN
player_location_location,"(1.9, 1.7, 1.2)",NaN,NaN,NaN


In [ ]:

mask_series = frvrs_logs_df.voice_command_command_description.isnull() & frvrs_logs_df.voice_capture_message.isnull()
df = frvrs_logs_df[~mask_series]
# df.sample(min(4, frvrs_logs_df[mask_series].shape[0])).dropna(axis='columns', how='all').T
df.voice_command_command_description.unique().tolist()

In [13]:

old_actions_list = [
    'SESSION_END', 'SESSION_START', 'BAG_ACCESS', 'BAG_CLOSED', 'INJURY_RECORD', 'INJURY_TREATED', 'PATIENT_DEMOTED', 'PATIENT_ENGAGED',
    'PATIENT_RECORD', 'PULSE_TAKEN', 'S_A_L_T_WALKED', 'S_A_L_T_WALK_IF_CAN', 'S_A_L_T_WAVED', 'S_A_L_T_WAVE_IF_CAN', 'TAG_APPLIED',
    'TAG_DISCARDED', 'TAG_SELECTED', 'TELEPORT', 'TOOL_APPLIED', 'TOOL_DISCARDED', 'TOOL_HOVER', 'TOOL_SELECTED', 'VOICE_CAPTURE',
    'VOICE_COMMAND'
]
missing_actions_list = list(set(frvrs_logs_df.action_type.unique()) - set(old_actions_list))
print(missing_actions_list)
print(f'@Jeremy Patterson, do you have an updated MCIVR Metrics Types document? I need {nu.conjunctify_nouns(missing_actions_list)}.')
for action in missing_actions_list:
    print(f'''        elif (action_type == '{action}'): # {''.join([x.title() for x in action.split('_')])}
            frvrs_logs_df.loc[row_index, '{action.lower()}_location'] = row_series[4] # Location
            frvrs_logs_df.loc[row_index, '{action.lower()}_patient_id'] = row_series[5] # patientId
            frvrs_logs_df.loc[row_index, '{action.lower()}_distance'] = row_series[7] # distance
            frvrs_logs_df.loc[row_index, '{action.lower()}_rotation'] = row_series[8] # rotation''')

['PLAYER_GAZE', 'PLAYER_LOCATION']
@Jeremy Patterson, do you have an updated MCIVR Metrics Types document? I need PLAYER_GAZE and PLAYER_LOCATION.
        elif (action_type == 'PLAYER_GAZE'): # PlayerGaze
            frvrs_logs_df.loc[row_index, 'player_gaze_location'] = row_series[4] # Location
            frvrs_logs_df.loc[row_index, 'player_gaze_patient_id'] = row_series[5] # patientId
            frvrs_logs_df.loc[row_index, 'player_gaze_distance'] = row_series[7] # distance
            frvrs_logs_df.loc[row_index, 'player_gaze_rotation'] = row_series[8] # rotation
        elif (action_type == 'PLAYER_LOCATION'): # PlayerLocation
            frvrs_logs_df.loc[row_index, 'player_location_location'] = row_series[4] # Location
            frvrs_logs_df.loc[row_index, 'player_location_patient_id'] = row_series[5] # patientId
            frvrs_logs_df.loc[row_index, 'player_location_distance'] = row_series[7] # distance
            frvrs_logs_df.loc[row_index, 'player_location_rotation'

In [95]:

mask_series = (frvrs_logs_df.voice_command_command_description == 'Jocelyn can you take a picture for Hannah')
mask_series |= (frvrs_logs_df.voice_capture_message == 'Jocelyn can you take a picture for Hannah')
df = frvrs_logs_df[mask_series]
columns_list = ['action_type', 'voice_capture_message']
df.sample(min(4, frvrs_logs_df[mask_series].shape[0])).dropna(axis='columns', how='all').T

,20287
action_type,VOICE_CAPTURE
elapsed_time,5163490
event_time,2023-05-16 17:41:00
session_uuid,724fdf45-3165-43b8-b9ca-dc07102d2886
file_name,724fdf45-3165-43b8-b9ca-dc07102d2886
logger_version,1.0
time_group,13
voice_capture_message,Jocelyn can you take a picture for Hannah


In [20]:

text_editor_path = r'"C:\Program Files\Notepad++\notepad++.exe"'
!{text_editor_path} "{os.path.abspath(file_path)}"

In [41]:

# Mask voice capture PII
columns_list = ['voice_command_command_description', 'voice_capture_message']
if not frvrs_logs_df[columns_list].applymap(lambda x: '[PERSON]' in str(x), na_action='ignore').sum().sum():
    import spacy
    nlp = spacy.load('en_core_web_sm')
    import en_core_web_sm
    nlp = en_core_web_sm.load()
    
    mask_series = frvrs_logs_df.voice_command_command_description.isnull() & frvrs_logs_df.voice_capture_message.isnull()
    df = frvrs_logs_df[~mask_series]
    def mask_pii(srs):
        for idx in columns_list:
            new_text = srs[idx]
            if str(new_text) != 'nan':
                doc = nlp(new_text)
                for entity in doc.ents:
                    if entity.label_ == 'PERSON':
                        new_text = re.sub('\\b' + entity.text + '\\b', '[PERSON]', new_text)
                srs[idx] = new_text
    
        return srs
    
    for row_index, row_series in df.apply(mask_pii, axis='columns')[columns_list].iterrows():
        for column_name, column_value in row_series.items():
            if str(column_value) != 'nan':
                frvrs_logs_df.loc[row_index, column_name] = column_value
    
    # nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    # nu.save_dataframes(frvrs_logs_df=frvrs_logs_df)

Bowser you know inclusive Bowser
joystick miss joystick
Michael Michael where are you hurt
Michael Michael where are you hurt
Michael Michael where are you hurt
Gloria are you Gloria
Gloria OK Gloria we'll take good care of you
Gloria where are you hurt alright Gloria
Gloria where are you hurt alright Gloria
Gloria where are you hurt alright Gloria
michael hi michael
god god
Michael oh hi Michael
Jocelyn Jocelyn can you take a picture for Hannah
Hannah [PERSON] can you take a picture for Hannah
Stiller take care of the person who's Stiller has an obvious life
Gray you said Gray
donna donna grab that that just comes with me
Marty so i need Marty here
dalia dalia that's much better
ben ham ben ham ben ham ben ham ben ham ben ham so i need to go do this guy
DRT alright he is DRT so we're gonna go ahead and yeah i'll get to you in a second
kinda i kinda looks like it's flowing but i don't think it looks like it might be inactive bleeding i can't tell
Jesus and stop there we go Jesus is gon